In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

In [3]:
meta_data = pd.read_csv('drive/MyDrive/FYP/Data/meta_data.csv')

In [4]:
def get_data(N):
    data = []
    
    path = f"{N}.npz"
    path = os.path.join("drive/MyDrive/FYP/Data/npz/", path)
    
    bat_files = np.load(path)
    bat_files.files
    
    for bat_file in bat_files['arr_0']:
        data.append(bat_file)
    
    return data

In [5]:
def internal_resistance(a, train_ratio = 0.2):
  """
  a : Battery ID.
  train_ratio : Training split ratio of data.
  """

  data = []
  data = get_data(N = a)

  # Time-stamps.
  t = []
  # Current Data.
  I = []
  # Voltage Data.
  V = []
  # Temperature Data.
  T = []
  
  for row in data:
    t.append(row[0])
    I.append(row[1])
    V.append(row[2])
    T.append(row[3])

  # Current, Voltage and Temperature Histogram.
  # plt.hist(I)
  # plt.hist(V)
  # plt.hist(T)
  
  # Data Truncation.
  X = np.atleast_2d(I[:5000]).T
  y = V[:5000]

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = train_ratio)
  
  # Create a Gaussian Process Regressor with a Wiener Velocity kernel
  kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0)) \
          + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-10, 1e+1))
  # Define the Gaussian process regressor with alpha=0 (no regularization)
  gp = GaussianProcessRegressor(kernel=kernel)

  # Fit the Gaussian process to the data
  gp.fit(X_train, y_train)

  # Estimate beta parameter
  beta = 1 / (2 * gp.alpha_)

  # Make predictions on the test data
  y_pred, sigma = gp.predict(X_test, return_std = True)

  # Plot the results
  # plt.figure()
  # plt.plot(X_test, y_test, 'r:', label=u'$f(x)$')
  # plt.plot(X_test, y_pred, 'b-', label=u'Prediction')
  # plt.fill(np.concatenate([X_test, X_test[::-1]]),
  #          np.concatenate([y_pred - 1.9600 * sigma,
  #                          (y_pred + 1.9600 * sigma)[::-1]]),
  #          alpha=.5, fc='b', ec='None')
  # plt.xlabel('$x$')
  # plt.ylabel('$f(x)$')
  # plt.ylim(-1, 20)
  # plt.legend(loc='upper left')
  # plt.show()

  mse = np.mean((y_test - y_pred) ** 2)
  print("Mean Squared Error:", mse)

  # Save Internal Resistance .npy file.
  np.save(f"drive/MyDrive/FYP/Internal_Resistance/{a}.npy", beta)

In [6]:
# for a in range(536, 587):
#   internal_resistance(a)

Mean Squared Error: 0.020346822971595437
Mean Squared Error: 0.12714103126816176
Mean Squared Error: 0.10698978394294138
Mean Squared Error: 0.1638490470694546
Mean Squared Error: 0.15345986079899468
Mean Squared Error: 0.10222604189050086
Mean Squared Error: 0.11686264178481426
Mean Squared Error: 0.08578577901113345
Mean Squared Error: 0.09639810285331307
Mean Squared Error: 0.058748367416072186
Mean Squared Error: 0.11389233026610548
Mean Squared Error: 0.08784921238424108
Mean Squared Error: 0.0601468583946193
Mean Squared Error: 0.06462008967276205
Mean Squared Error: 0.20661790671169097
Mean Squared Error: 0.06672883614496536
Mean Squared Error: 0.10601498685546386
Mean Squared Error: 0.1303546066484
Mean Squared Error: 0.05513414112640015
Mean Squared Error: 0.039817595817356714
Mean Squared Error: 0.1421903211711188
Mean Squared Error: 0.24172157793402232
Mean Squared Error: 0.12536300131910072
Mean Squared Error: 0.055688072168643854
Mean Squared Error: 0.17315130599771322
Mea

In [7]:
 labels = []

for label in meta_data["STILL_ALIVE"]:
  if label == True:
    labels.append(1)
  else:
    labels.append(0)

In [11]:
# Internal Resistances.
R = []
# New Labels.
U = []


for a in range(101):
  R.append(
      np.load(f"drive/MyDrive/FYP/Internal_Resistance/{a}.npy")
  )
  U.append(labels[a])


for a in range(536, 587):
  R.append(
      np.load(f"drive/MyDrive/FYP/Internal_Resistance/{a}.npy")
  )
  U.append(labels[a])

In [30]:
R_train, R_test, U_train, U_test = train_test_split(R, U, test_size = 0.5)

In [31]:
# SVM using Gaussian Kernel.

clf = svm.SVC(kernel="rbf")
clf.fit(R_train, U_train)

SVC()

In [32]:
clf.predict(R_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])